### Markdown example
Blab bla documentation


In [3]:
import requests
import untangle
import pprint

pp = pprint.PrettyPrinter(indent=4)

r = requests.get('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=212403', auth=('user', 'pass'))
print r
print ''
print 'Status code: ' 
print r.status_code
print ''
print 'Headers: '
print r.headers
print ''


xmlObj = untangle.parse(r.text)

pmid_lst = []

for ref in xmlObj.pmc_articleset.article.back.ref_list.ref:
    this_cite = ref.citation
    
    if hasattr(this_cite, 'pub_id'):
        pubs = this_cite.pub_id
        if type(pubs) == list:
            for pub_el in pubs:
                if pub_el['pub-id-type'] == 'pmid':
                    #print 'Reference: '
                    #print this_cite.article_title.cdata
                    #print 'Entry type:'
                    #print type(pub_el)
                    #print 'Has PMID:'
                    #print pub_el.cdata
                    pmid_lst.append(pub_el.cdata)
        elif pubs['pub-id-type'] == 'pmid':
            #print 'Reference: '
            #print this_cite.article_title.cdata
            #print 'Entry type:'
            #print type(pubs)
            #print 'Has PMID:'
            #print pubs.cdata
            pmid_lst.append(pubs.cdata)
            
print 'All cited PMIDs:'
print pmid_lst
print len(pmid_lst)

<Response [200]>

Status code: 
200

Headers: 
{'X-XSS-Protection': '1; mode=block', 'NCBI-PHID': '03ED5B8C816250710000000003D303C0', 'Content-Security-Policy': 'upgrade-insecure-requests', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Set-Cookie': 'ncbi_sid=03ED5B8C81630B51_0979SID; domain=.nih.gov; path=/; expires=Mon, 30 Oct 2017 17:41:09 GMT', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'Vary': 'Accept-Encoding', 'Keep-Alive': 'timeout=1, max=10', 'Server': 'Apache', 'Content-Security-Policy-Report-Only': "default-src https:; script-src https: 'unsafe-inline' 'unsafe-eval'; font-src https: data:; img-src https: data:; style-src https: 'unsafe-inline'; report-uri https://www.ncbi.nlm.nih.gov/corecgi/csp/csp.cgi", 'NCBI-SID': '03ED5B8C81630B51_0979SID', 'Connection': 'Keep-Alive', 'X-UA-Compatible': 'IE=Edge', 'Cache-Control': 'private', 'Date': 'Sun, 30 Oct 2016 17:41:09 GMT', 'Access-Control-Allow-Origin': '*', 'Content-Type': 'text/x

In [1]:
import sys
sys.path.append('../lib')
from ConnectEutils import ConnectEutils


eutils = ConnectEutils()

eutils.get_cited_PMID('212403')

[u'8216567',
 u'7819893',
 u'12402719',
 u'11173046',
 u'12118235',
 u'12020178',
 u'12020180',
 u'12020179',
 u'12609941',
 u'9870574',
 u'11158556',
 u'10985878',
 u'10584742',
 u'8880846',
 u'8721797',
 u'8880845',
 u'9085306',
 u'9060022',
 u'7873954',
 u'10781914',
 u'11822262',
 u'10812319',
 u'11106885',
 u'9310564',
 u'3869471',
 u'3540871',
 u'10485502',
 u'8269451',
 u'7923306',
 u'1106936',
 u'11198829',
 u'10431716',
 u'10700324',
 u'9150293',
 u'10641469']